In [ ]:
import torch
from torchvision import models 
import torchvision.transforms as T
from torchvision.transforms.functional import InterpolationMode
from PIL import Image
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
import numpy.ma as ma
import cv2
from scipy.special import softmax
import os
import shutil
import time
fcn = models.segmentation.fcn_resnet101(pretrained=True).eval()
dlab = models.segmentation.deeplabv3_resnet101(pretrained=1).eval()
model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=True)


In [ ]:
TRAIN_FOLDER = "/kaggle/input/happy-whale-and-dolphin/train_images/"
TEST_FOLDER = "/kaggle/input/happy-whale-and-dolphin/test_images/"
HEIGHT = 244
WIDTH = 244
THRESHOLD = 0.05
# Import the training data
traindata = pd.read_csv("/kaggle/input/happy-whale-and-dolphin/train.csv")
traindata.loc[:,'image'] = TRAIN_FOLDER + traindata['image']
# Import the test data
testdata = pd.read_csv("/kaggle/input/happy-whale-and-dolphin/sample_submission.csv")
testdata.loc[:,'image'] = TEST_FOLDER + testdata['image']

outputdir = '/kaggle/working/'

if not os.path.exists(outputdir+"/train_images/"):
    os.makedirs(outputdir+"/train_images/")
    
if not os.path.exists(outputdir+"/test_images/"):
    os.makedirs(outputdir+"/test_images/")
    

original = r'/kaggle/input/happy-whale-and-dolphin/train.csv'
target = r'./train.csv'
shutil.copyfile(original, target)

original = r'/kaggle/input/happy-whale-and-dolphin/sample_submission.csv'
target = r'./sample_submission.csv'
shutil.copyfile(original, target)

In [ ]:
# Define the helper function
def decode_segmap(image, nc=2):
  
    label_colors = np.array([(0, 0, 0),  # 0=background
               # 1=aeroplane, 2=bicycle, 3=bird, 4=boat, 5=bottle
               (128, 0, 0), (0, 128, 0), (128, 128, 0), (0, 0, 128), (128, 0, 128),
               # 6=bus, 7=car, 8=cat, 9=chair, 10=cow
               (0, 128, 128), (128, 128, 128), (64, 0, 0), (192, 0, 0), (64, 128, 0),
               # 11=dining table, 12=dog, 13=horse, 14=motorbike, 15=person
               (192, 128, 0), (64, 0, 128), (192, 0, 128), (64, 128, 128), (192, 128, 128),
               # 16=potted plant, 17=sheep, 18=sofa, 19=train, 20=tv/monitor
               (0, 64, 0), (128, 64, 0), (0, 192, 0), (128, 192, 0), (0, 64, 128)])

    label_colors = np.array([(1,1,1)]*2)
    label_colors[0] = (0,0,0)
    
    r = np.zeros_like(image).astype(np.uint8)
    g = np.zeros_like(image).astype(np.uint8)
    b = np.zeros_like(image).astype(np.uint8)
  
    for l in range(0, nc):
        idx = image == l
        r[idx] = label_colors[l, 0]
        g[idx] = label_colors[l, 1]
        b[idx] = label_colors[l, 2]

        rgb = np.stack([r, g, b], axis=2)
    return rgb

def masked_image(net, path):
    #img = crop(yolo, path)
    #img = Image.fromarray(np.uint8(img))
    img = Image.open(path)
    # Comment the Resize and CenterCrop for better inference results
    trf = T.Compose([T.Resize(105,interpolation=InterpolationMode.NEAREST),
                   #T.CenterCrop(300),
                   T.ToTensor(),
                   T.Normalize(mean = [0.485, 0.456, 0.406],
                               std = [0.229, 0.224, 0.225])])
    try:
        inp = trf(img).unsqueeze(0)
    except RuntimeError:
        return np.array(img)
    out = dlab(inp)['out'].squeeze().detach().numpy() #Lag point.

    out = softmax(out, axis=0)[0]  
    out = 1-out
    #img = cv2.resize(np.array(img), (HEIGHT,WIDTH), interpolation = cv2.INTER_AREA)
    out = cv2.resize(out, img.size, interpolation = cv2.INTER_LINEAR)
    out[out < THRESHOLD] = 0
    out[out >= THRESHOLD] = 1
    if np.sum(out)<1:
        return np.array(img)
    else:
        out = np.reshape(out, (out.shape[0], out.shape[1], 1))
        return (out*np.array(img)/255.0)
    
def progress_bar(perc_comp):
    '''
    Takes float or int between 0 and 100. 
    '''
    bar_width = 100
    bar = '#' * int(perc_comp)
    bar += '-' * (100-int(perc_comp)) + ' ' + str(int(perc_comp)) + '% complete'
    if perc_comp < 100:
        print (bar, end='\r')
    else:
        print (bar)

In [ ]:
plt.imshow(masked_image(dlab, traindata.image.values[0]))

In [ ]:
for i in traindata.image.values[0:10]:
    fig, ax = plt.subplots(1,2, figsize=(10,10))
    mask = masked_image(dlab, i)
    ax[0].imshow(mask)
    ax[1].imshow(Image.open(i))
    plt.show()

In [ ]:
t1 = time.time()
for i in range(len(traindata.image.values)):
    output = outputdir+"/test_images/"+traindata.image.values[i].split('/')[-1]
    img = masked_image(dlab, traindata.image.values[i])*255.0
    #im = cv2.resize(im, (244,244), interpolation = cv2.INTER_AREA)
    cv2.imwrite(output, img)
    progress_bar(i/(len(traindata.image.values)-1)*100)   
t2 = time.time()
print(f'Executed in {(t2-t1):.4f}s')